# Demoing fetching PV locations from DuckLake into a LanceDB dataset



### References:
- [DuckLake Documentation](https://ducklake.select/docs/stable/)
- [DuckLake with Ibis Python DataFrames](https://emilsadek.com/blog/ducklake-ibis/)
- [A new data lakehouse with DuckLake and dbt](https://giacomo.coletto.io/blog/ducklake/)

In [1]:
import ibis
import duckdb
import pandas as pd

from tqdm import tqdm
from dotenv import load_dotenv

import os
import shutil

In [2]:
ibis.options.interactive = True
ibis.options.graphviz_repr=True

load_dotenv()

# assume we're using prod catalog, but default to local/dev if env var not set
local_default = os.getenv('DUCKLAKE_CONNECTION_STRING_DEV')
DUCKLAKE_CATALOG = os.getenv('DUCKLAKE_CONNECTION_STRING_PROD', local_default)
DUCKLAKE_ATTACH = os.getenv("DUCKLAKE_ATTACH_PROD")
DUCKLAKE_NAME = os.getenv("DUCKLAKE_NAME")
DUCKLAKE_DATA_PATH = os.getenv("DUCKLAKE_DATA_PATH")

# pretty print our connection string info 
# TODO: comment out and remove output before commit
print(f"Using DuckLake catalog type: {DUCKLAKE_CATALOG.split(':')[1]}" )
catalog_creds = DUCKLAKE_CATALOG.split(':')[2].strip('()').split(' ')
# skip DATA_PATH at end
for cred in catalog_creds[:-2]:
    key, val = cred.split('=')
    print(f"  {key}: {val}")
print(f"  DATA_PATH: {DUCKLAKE_CATALOG.split('DATA_PATH ')[1][:-1]}")

Using DuckLake catalog type: postgres
  dbname: ducklake_catalog
  host: ep-broad-rain-a4tdwnxn-pooler.us-east-1.aws.neon.tech
  user: neondb_owner
  password: npg_y0ViUzD1Xulc
  port: 5432
  sslmode: require
  DATA_PATH: 's3://eo-pv-lakehouse/ducklake_data'


### Connect to our data lake catalog with ibis 



In [3]:

con = ibis.duckdb.connect(extensions=["ducklake", "spatial", "httpfs"])

r2_bucket_setup = f"""SET s3_access_key_id='{os.getenv('R2_ACCESS_KEY_ID')}';
SET s3_secret_access_key='{os.getenv('R2_SECRET_KEY')}';
SET s3_endpoint='{os.getenv('R2_S3_ENDPOINT')}';
SET s3_use_ssl='true';
SET s3_url_style='path';
"""
con.raw_sql(r2_bucket_setup)

In [4]:
# con.load_extension("ducklake")
attach_catalog_sql = f"""ATTACH IF NOT EXISTS '{DUCKLAKE_ATTACH}' AS {DUCKLAKE_NAME}
    (DATA_PATH '{DUCKLAKE_DATA_PATH}');
USE {DUCKLAKE_NAME};
"""
con.raw_sql(attach_catalog_sql)
con.list_catalogs()

['__ducklake_metadata_eo_pv_lakehouse',
 'eo_pv_lakehouse',
 'memory',
 'system',
 'temp']

In [5]:
con.list_tables()

['pv_h3_cells',
 'pv_h3_grid',
 'raw_chn_med_res_pv_2024',
 'raw_global_harmonized_large_solar_farms_2020',
 'raw_global_pv_inventory_sent2_spot_2021',
 'raw_ind_pv_solar_farms_2022',
 'raw_uk_crowdsourced_pv_2020',
 'raw_usa_cali_usgs_pv_2016',
 'stg_chn_med_res_pv_2024',
 'stg_global_harmonized_large_solar_farms_2020',
 'stg_global_pv_inventory_sent2_spot_2021',
 'stg_ind_pv_solar_farms_2022',
 'stg_pv_consolidated',
 'stg_uk_crowdsourced_pv_2020',
 'stg_usa_cali_usgs_pv_2016']

In [6]:
stg_pv = con.table("stg_pv_consolidated")
stg_pv.head()

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━┳━━━━━━━━━━━━━━┓
┃ dataset_name                        ┃ processed_at                  ┃ geometry                                                                         ┃ area_m2      ┃ centroid_lat ┃ centroid_lon ┃ h3_index_8         ┃ unified_id ┃ source_area_m2 ┃ capacity_mw ┃ install_date ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━╇━━━━━━━━━━━━━━┩
│ string                              │ timestamp('UTC', 6)           │ string                                                                           │ float64      │ float64      │ float64      │ int64              │ string     │ float64        │ float64     │ string       │
├─────────────────────────────────────┼───────────────────────────────┼──────────────────────────────────────────────────────────────────────────────────┼──────────────┼──────────────┼──────────────┼────────────────────┼────────────┼────────────────┼─────────────┼──────────────┤
│ global_pv_inventory_sent2_spot_2021 │ 2025-09-26 03:24:11.103+00:00 │ POLYGON ((21.614604005422446 38.11466204732934, 21.614661136097887 38.114626492… │ 21958.162864 │    38.115039 │    21.615342 │ 613600967155253247 │ 0.0        │   13592.361843 │    1.201704 │ ~            │
│ global_pv_inventory_sent2_spot_2021 │ 2025-09-26 03:24:11.103+00:00 │ POLYGON ((6.379356223001148 50.19216226866618, 6.379386869088315 50.19213902072… │  8323.457214 │    50.191844 │     6.379806 │ 613046147732209663 │ 1.0        │    3411.622302 │    0.309810 │ ~            │
│ global_pv_inventory_sent2_spot_2021 │ 2025-09-26 03:24:11.103+00:00 │ POLYGON ((-0.896474365596765 52.03906610129858, -0.896514092788355 52.039104844… │ 55207.651881 │    52.038515 │    -0.895182 │ 612935798196535295 │ 2.0        │   20889.852876 │    1.752301 │ ~            │
│ global_pv_inventory_sent2_spot_2021 │ 2025-09-26 03:24:11.103+00:00 │ POLYGON ((21.655447685837466 38.108830922166376, 21.655447082367587 38.10876338… │  6710.398589 │    38.108923 │    21.655906 │ 613600967341899775 │ 3.0        │    4154.511624 │    0.397576 │ ~            │
│ global_pv_inventory_sent2_spot_2021 │ 2025-09-26 03:24:11.103+00:00 │ POLYGON ((14.279220062197856 50.517537062657965, 14.279242655716823 50.51744282… │ 11858.756553 │    50.517348 │    14.279998 │ 613021008586080255 │ 4.0        │    4794.474966 │    0.440007 │ ~            │
└─────────────────────────────────────┴───────────────────────────────┴──────────────────────────────────────────────────────────────────────────────────┴──────────────┴──────────────┴──────────────┴────────────────────┴────────────┴────────────────┴─────────────┴──────────────┘

In [7]:
from ibis import _
stg_pv.aggregate(by=["h3_index_8"], pv_area=_.area_m2.sum()).order_by(ibis.desc("pv_area")).head()

┏━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━┓
┃ h3_index_8         ┃ pv_area      ┃
┡━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━┩
│ int64              │ float64      │
├────────────────────┼──────────────┤
│ 613134297504677887 │ 3.778312e+08 │
│ 613133048860377087 │ 2.476448e+08 │
│ 613132382727307263 │ 2.232192e+08 │
│ 613132382718918655 │ 1.814395e+08 │
│ 613139556146872319 │ 1.419494e+08 │
└────────────────────┴──────────────┘

In [9]:
# load h3 duckdb extension and calculate coarser h3 index
con.load_extension("h3")
# test returning column of h3 cells 
stg_pv.sql("SELECT h3_cell_to_parent(h3_index_8, 6) FROM stg_pv_consolidated LIMIT 10").head()

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ h3_cell_to_parent(h3_index_8, 6) ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ int64                            │
├──────────────────────────────────┤
│               604593768015855615 │
│               604038948569743359 │
│               603928599048749055 │
│               604593768150073343 │
│               604013809455071231 │
└──────────────────────────────────┘

## Hugging Face Hub

The [Hugging Face Hub](https://huggingface.co/docs/hub/index) is a Machine Learning platform with hundreds of thousands of open source and publicly available datasets, models, and interactive model demos. 


## Google S2 Geometric Spatial Indexing

See python S2 library: [S2Cell Docs](https://docs.s2cell.aliddell.com/en/stable/index.html)

See details: https://learn.microsoft.com/en-us/kusto/query/geo-point-to-s2cell-function?view=microsoft-fabric

In [ ]:
# Querying (remote) Hub files with DuckDB

from huggingface_hub import HfFileSystem
import duckdb

fs = HfFileSystem()
duckdb.register_filesystem(fs)
# Query a remote file and get the result back as a dataframe
fs_query_file = "hf://datasets/my-username/my-dataset-repo/data_dir/data.parquet"
df = duckdb.query(f"SELECT * FROM '{fs_query_file}' LIMIT 10").df()

In [ ]:
# temp for quick dev iteration: use local geoparquet export of stg_pv_consolidated